In [2]:
import pandas as pd 
import os 

In [7]:
train_data = pd.read_csv("./data/dataset/raw_data_pinterest/train/_annotations.csv")
train_data.head() 

,filename,width,height,class,xmin,ymin,xmax,ymax
0,59ed27db4b561f28eb48c380b8940573_jpg.rf.dee95c...,416,416,top,103,149,399,410
1,59ed27db4b561f28eb48c380b8940573_jpg.rf.dee95c...,416,416,glasses,162,99,254,125
2,b911cc29e6614dff3bb3d5e01387bd0b_jpg.rf.dcc99e...,416,416,top,113,142,287,280
3,b911cc29e6614dff3bb3d5e01387bd0b_jpg.rf.dcc99e...,416,416,pants,101,1,282,139
4,4b72224fbd21490c283c25f7881b08b0--blazer-casua...,416,416,pants,224,246,386,414


In [8]:
val_data = pd.read_csv("./data/dataset/raw_data_pinterest/valid/_annotations.csv") 
val_data.shape

(83, 8)

In [11]:
train_data = pd.concat([train_data, val_data]).reset_index(drop=True)

In [12]:
test_data = pd.read_csv("./data/dataset/raw_data_pinterest/test/_annotations.csv") 

In [34]:
test_data.shape 

(7, 8)

In [26]:
classes = {value:key for (key,value) in train_data['class'].value_counts().index.to_series().reset_index(drop=True).to_dict().items()}
classes

{'shoes': 0,
 'top': 1,
 'pants': 2,
 'bag': 3,
 'glasses': 4,
 'accessories': 5,
 'hat': 6,
 'dress': 7,
 'low': 8}

In [27]:
def parse_csv_to_pascal(df: pd.DataFrame): 
    data = {} 

    for i in range(len(df.index)):
        xmin = int(df.iloc[i]['xmin'])
        ymin = int(df.iloc[i]['ymin'])
        xmax = int(df.iloc[i]['xmax'])
        ymax = int(df.iloc[i]['ymax']) 

        object_class = classes[df.iloc[i]['class']]

        if df.iloc[i]['filename'] in data.keys():
            data[df.iloc[i]['filename']].append([xmin, ymin, xmax, ymax, object_class])
        else:
            data.update({df.iloc[i]['filename'] : [[xmin, ymin, xmax, ymax, object_class]]})
    

    return data 

In [28]:
train_pascal_dic = parse_csv_to_pascal(train_data)

In [32]:
test_pascal_dic = parse_csv_to_pascal(test_data) 

In [33]:
import shutil 

def modify_data(data, path_to_images, path_to_output): 
    path_to_save_annotations = os.path.join(path_to_output, 'annotations.txt') 

    with open(path_to_save_annotations, 'a+') as f:

        for img_name, detections in data.items(): 

            path_to_input_img = os.path.join(path_to_images, img_name) 

            name_without_spaces = img_name.replace(' ','') 
            path_to_output_img = os.path.join(path_to_output, name_without_spaces)

            shutil.copy(path_to_input_img, path_to_output_img)

            # save detections in the new annotations file
            f.write(f'{path_to_output_img} ')
            for detection in detections:
                xmin, ymin, xmax, ymax, c = detection
                f.write(f'{xmin},{ymin},{xmax},{ymax},{c} ')
            
            f.write('\n')

    print('Done saving annotations')

In [36]:
modify_data(train_pascal_dic, './data/dataset/raw_data_pinterest/train/', './data/dataset/processed_yolo/train/')

Done saving annotations


In [37]:
modify_data(test_pascal_dic, './data/dataset/raw_data_pinterest/test/', './data/dataset/processed_yolo/test/')

Done saving annotations
